### konlpy 사용 형태소 분석

In [3]:
from konlpy.tag import Twitter; tw = Twitter()

song = '동해물과 백 두산이 마르고 닳도록 하느님이 보우하사 우리나라 만세. 무궁화 삼철리 화려강산 대한 사람, 대한으로 길이 보전하세'
word_count = dict()
words = tw.nouns(song)
for word in words:
    if word in word_count:
        word_count[word]+= 1
    else:
        word_count[word]=1
        
word_count

{'강산': 1,
 '길이': 1,
 '닳': 1,
 '대한': 2,
 '도록': 1,
 '동해': 1,
 '두산': 1,
 '마르고': 1,
 '만세': 1,
 '무궁화': 1,
 '물': 1,
 '백': 1,
 '보우': 1,
 '보전': 1,
 '사람': 1,
 '삼': 1,
 '우리나라': 1,
 '철리': 1,
 '하느님': 1,
 '하사': 1,
 '화려': 1}

### Clustering-Norm 2

In [6]:
import math

Doc1 = {'term1':1, 'term2':1, 'term3':1, 'term4':0, 'term5':0}
Doc2 = {'term1':0, 'term2':0, 'term3':2, 'term4':2, 'term5':1}
Doc3 = {'term1':2, 'term2':2, 'term3':2, 'term4':0, 'term5':1}

Doc1_euclidean_norm = 0
sum_value = 0
for j in Doc1:
    sum_value += math.pow(Doc1[j],2) # 모든 단어에 대한 제곱값을 sum_value에 누적해 더한다.
Doc1_euclidean_norm = math.sqrt(sum_value) # sum_value에 루트를 씌워서, 기준값을 완성한다.
print('Doc1 euclidean norm :', Doc1_euclidean_norm)

Doc1_normalize= []
for i in Doc1:
    Doc1_normalize.append(Doc1[i]/Doc1_euclidean_norm) # 각 term들에 대해 기준값으로 나눠 단어에 대한 점수를 구한다
print('Doc1 normalize:', Doc1_normalize)

Doc1 euclidean norm : 1.7320508075688772
Doc1 normalize: [0.0, 0.5773502691896258, 0.0, 0.5773502691896258, 0.5773502691896258]


In [16]:
import numpy as np

Doc1 = np.array([1,1,1,0,0])
Doc2 = np.array([0,0,2,1,1])
Doc3 = np.array([2,2,2,0,1])

#내적
Doc1_dot_Doc2 = np.dot(Doc1, Doc2)
print('doc1, doc2 (inner product) : ', Doc1_dot_Doc2)
Doc1_dot_Doc3 = np.dot(Doc2, Doc3)
print('doc1, doc3 (inner product) : ', Doc1_dot_Doc3)
Doc2_dot_Doc3 = np.dot(Doc2, Doc3)
print('doc2, doc3 (inner product) : ', Doc2_dot_Doc3)

# COS 계산
print(1- (Doc1_dot_Doc2/ (np.sqrt(np.sum(np.power(Doc1, 2))) *
                         np.sqrt(np.sum(np.power(Doc2,2))))))
      
print(1- (Doc1_dot_Doc3/ (np.sqrt(np.sum(np.power(Doc1, 2))) *
                         np.sqrt(np.sum(np.power(Doc3,2))))))

print(1- (Doc1_dot_Doc3/ (np.sqrt(np.sum(np.power(Doc2, 2))) *
                         np.sqrt(np.sum(np.power(Doc3,2))))))

doc1, doc2 (inner product) :  2
doc1, doc3 (inner product) :  5
doc2, doc3 (inner product) :  5
0.5285954792089682
0.1993592309745642
0.4338614829277021


### TF-IDF (Term Frequency - Inverse Frequency
- 문서의 중요한 단어를 찾기 위해

In [98]:
import math
from collections import Counter

docu = {'doc_a' : ['딸기', '안녕', '안녕', '안녕'],
        'doc_b' : ['안녕', '안녕', '시금치'],
        'doc_c' : ['딸기', '안녕', '사슴', '사슴'],
        'doc_d' : ['딸기', '안녕'],
        'doc_e' : ['시금치', '안녕']}

total_tf_idf = {}

for d in docu.keys():
    ct = Counter(docu[d])
    # 문서에 나타난 빈도 또는 특정한 단어의 빈도수를 전체 단어수로 나눠 나온 단어별 비율
    
    ### tf  ###
    tf = {}
    for keyd in ct.keys():
        tf[keyd] = ct[keyd]/ len(docu[d])
        
    ### df  ###
    # dict 초기화
    df = {}
    for cnoun in ct.keys():
        df[cnoun] = 0 
        
    # 전체 문서에서 특정 키워드가 나타나는 횟수
    for cnoun in ct.keys():
        for keyword in docu.keys():
            
            # 특정키워드가 있는지를 파악
            sp_keyword = set(docu[keyword]) & {cnoun}  # () 안에들어갈 연사자는?  집합간의 연산을 할떄는 & 또는 | 를 사용   

            # 특정 키워드가 있으면 1씩 카운트 상승
            if sp_keyword != set():
                df[cnoun] += 1

    tf_idf = {}
    for keyword in ct.keys():
        #### IDF ####
        idf = math.log((len(docu.keys())+1)/ (1+df[keyword]))  # +1을 해준것은 0이 되는것을 방지하기 위해
#         idf = math.log(len(docu.keys())/(1+df[keyword]))   # 이 식도 사용이 가능하지만 정보량이 작아 -값이 나오는것
                                                            # -가 나올땐 0으로 치환해주는 과정 필요
        
        ### TF * IDF  ###
        tf_idf[keyword] = tf[keyword] * idf
    total_tf_idf[d] = tf_idf

### 각 문서에서의 단어의 중요도

In [99]:
total_tf_idf

{'doc_a': {'딸기': 0.1013662770270411, '안녕': 0.0},
 'doc_b': {'시금치': 0.23104906018664842, '안녕': 0.0},
 'doc_c': {'딸기': 0.1013662770270411, '사슴': 0.5493061443340549, '안녕': 0.0},
 'doc_d': {'딸기': 0.2027325540540822, '안녕': 0.0},
 'doc_e': {'시금치': 0.34657359027997264, '안녕': 0.0}}

In [93]:
np.log(len(docu.keys())/(1+df['안녕']))

-0.1823215567939546

### Word tokenizing

### L Tokenizer 

### Soynlp

In [124]:
# import soynlp
from pprint import pprint
from soynlp.tokenizer import MaxScoreTokenizer, RegexTokenizer, LTokenizer

### MaxTokenizer

In [116]:
scores = {'파스':0.3, '파스타':0.7, '좋아요':0.2, '좋아':0.5} # 스코어 지정
tokenizer = MaxScoreTokenizer(scores=scores)  # 스코어가 높은것을 기준으로 뽑는 MaxTokenizer
tokenizer.tokenize('파스타가좋아요')

['파스타', '가', '좋아', '요']

In [120]:
# 띄어쓰기가 있는 문장
print('flatten=False')
pprint(tokenizer.tokenize("난파스타가 좋아요", flatten=False))   # (subword, begin, end, score, 글자수  순)

print('\nflatten=True')
pprint(tokenizer.tokenize('난파스타가 좋아요'))

flatten=False
[[('난', 0, 1, 0.0, 1), ('파스타', 1, 4, 0.7, 3), ('가', 4, 5, 0.0, 1)],
 [('좋아', 0, 2, 0.5, 2), ('요', 2, 3, 0.0, 1)]]

flatten=True
['난', '파스타', '가', '좋아', '요']


### LTokenizer

In [126]:
from soynlp.tokenizer import LTokenizer
scores = {'데이':0.5, '데이터':0.5, '데이터마이닝':0.5, '공부':0.5, '공부중':0.45}

tokenizer = LTokenizer(scores=scores)

print('\flatten = True \n sent = 데이터마이닝을 공부한다')
print(tokenizer.tokenize('데이터마이닝을 공부한다'))

print('\flatten = True \n sent = 데이터마이닝을 공부한다')
print(tokenizer.tokenize('데이터마이닝을 공부한다', flatten = False))

print('\flatten = True \n sent = 데이터분석을 위해서 데이터마이닝을 공부한다')
print(tokenizer.tokenize('데이터분석을 위해서 데이터마이닝을 공부한다', flatten = False))

latten = True 
 sent = 데이터마이닝을 공부한다
['데이터마이닝', '을', '공부', '한다']
latten = True 
 sent = 데이터마이닝을 공부한다
[('데이터마이닝', '을'), ('공부', '한다')]
latten = True 
 sent = 데이터분석을 위해서 데이터마이닝을 공부한다
[('데이터', '분석을'), ('위해서', ''), ('데이터마이닝', '을'), ('공부', '한다')]


In [130]:
print('tolerance = 0.0\nsent = 데이터마이닝을 공부중이다')
print(tokenizer.tokenize('데이터마이닝을 공부중이다'))

print('\ntolerance = 0.1\nsent = 데이터마이닝을 공부중이다')
print(tokenizer.tokenize('데이터마이닝을 공부중이다', tolerance=0.1))  
# tolerance (관용, 허용오차)를 주면 너무 빡빡하게 단어를 나누는게 아니고 어느정도 허용어차를 주고 단어선택을 한다는 뜻

tolerance = 0.0
sent = 데이터마이닝을 공부중이다
['데이터마이닝', '을', '공부', '중이다']

tolerance = 0.1
sent = 데이터마이닝을 공부중이다
['데이터마이닝', '을', '공부중', '이다']


- tolerance 는 기존의 스코어에 + 느낌으로 허용해준다고 보면 될듯

In [136]:
scores = {'데이':0.5, '데이터':0.5, '데이터마이닝':0.5, '공부':0.6, '공부중':0.45, '분석':0.9}

tokenizer = LTokenizer(scores=scores)

print('\ntolerance=0.1\nsent=데이터마이닝을 공부중이다')
print(tokenizer.tokenize('데이터마이닝을 공부중이다', tolerance=0.1))


tolerance=0.1
sent=데이터마이닝을 공부중이다
['데이터마이닝', '을', '공부', '중이다']


### RegexTozenizer
- 여러 언어가 쓰일때(한국 -> 외래어 -> 숫자등) 그 지점에서 토크나이징 됨

In [133]:
tokenizer = RegexTokenizer()

sents = [
    '이렇게연속된문장은잘리지않습니다만',
    '숫자123이영어abc에섞여있으면ㅋㅋ잘리겠죠',
    '띄어쓰기가 포함되어있으면 이정보는10점!꼭띄워야죠'
]

for sent in sents:
    print('     %s\n -> %s\n'  % (sent, tokenizer.tokenize(sent)))

     이렇게연속된문장은잘리지않습니다만
 -> ['이렇게연속된문장은잘리지않습니다만']

     숫자123이영어abc에섞여있으면ㅋㅋ잘리겠죠
 -> ['숫자', '123', '이영어', 'abc', '에섞여있으면', 'ㅋㅋ', '잘리겠죠']

     띄어쓰기가 포함되어있으면 이정보는10점!꼭띄워야죠
 -> ['띄어쓰기가', '포함되어있으면', '이정보는', '10', '점', '!', '꼭띄워야죠']



In [137]:
# flatten = False 띄어쓰기 기준으로 토큰을 나눠서 출력한다
tokenizer.tokenize('띄어쓰기가 포함되어 있으면 이정보는 10점!꼭띄워야죠', flatten=False)

[('띄어쓰기가', ''), ('포함되어', ''), ('있으면', ''), ('이정보는', ''), ('10점!꼭띄워야죠', '')]

In [121]:
def ltokenize(w):
    n = len(w)
    if n <= 2: return(w,'')
    tokens = []
    for e in range(2, n+1):
        tokens.append(w[:e], w[e:], cohesion(w[:e]))
    tokens = sorted(tokens, key=lambda x:-x[2])
    return tokens[0][:2]
sent = '뉴스의 기사를 이용했던 예시입니다'
for word in sent.split():
    print(ltokenize(word))

NameError: name 'cohesion' is not defined